In [ ]:
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
from google.colab import drive


In [ ]:
drive.mount('/content/drive'),

Mounted at /content/drive


(None,)

In [ ]:
train_dir = '/content/drive/MyDrive/Blood_Cell_Image_Dataset/Train'
test_dir = '/content/drive/MyDrive/Blood_Cell_Image_Dataset/Test'

In [ ]:

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [ ]:
train_images = train_gen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_gen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_gen.flow_from_directory(
    directory=test_dir,
    target_size=(224, 224),
    color_mode='rgb',
    batch_size=32,
    shuffle=False,
)

Found 5260 images belonging to 8 classes.
Found 1312 images belonging to 8 classes.
Found 439 images belonging to 1 classes.


In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
inputs = pretrained_model.input
x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
outputs = tf.keras.layers.Dense(8, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


Training

In [ ]:
Model_History = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
165/165 [==============================] - 900s 5s/step - loss: 0.9825 - accuracy: 0.6667 - val_loss: 0.6818 - val_accuracy: 0.7515
Epoch 2/100
165/165 [==============================] - 330s 2s/step - loss: 0.6048 - accuracy: 0.7905 - val_loss: 0.5982 - val_accuracy: 0.7820
Epoch 3/100
165/165 [==============================] - 307s 2s/step - loss: 0.5259 - accuracy: 0.8169 - val_loss: 0.5461 - val_accuracy: 0.8026
Epoch 4/100
165/165 [==============================] - 304s 2s/step - loss: 0.4738 - accuracy: 0.8338 - val_loss: 0.5660 - val_accuracy: 0.7965
Epoch 5/100
165/165 [==============================] - 301s 2s/step - loss: 0.4347 - accuracy: 0.8418 - val_loss: 0.5344 - val_accuracy: 0.8064
Epoch 6/100
165/165 [==============================] - 307s 2s/step - loss: 0.4042 - accuracy: 0.8565 - val_loss: 0.5193 - val_accuracy: 0.8140
Epoch 7/100
165/165 [==============================] - 306s 2s/step - loss: 0.3761 - accuracy: 0.8688 - val_loss: 0.5057 - val_accuracy:

In [ ]:
fig = px.line(
    Model_History.history,
    y=['loss', 'val_loss'],
    labels={'index': "Epoch", 'value': "Loss"},
    title="Training and Validation Loss Over Time"
)

fig.show()

In [ ]:
Name_Of_Class = list(train_images.class_indices.keys())
Name_Of_Class

['0', '1', '2', '3', '4', '5', '6', '7']

In [ ]:
predictions = model.predict(test_images)

In [ ]:
import os
from keras.models import model_from_json

Prediction_result = model.to_json()
with open("Group.json", "w") as json_file:
    json_file.write(Prediction_result)
model.save_weights("CorDweights.h5")
print("model saved to disk....")

In [ ]:
val_images.reset()
pred = model.predict_generator(generator=val_images, 
                               steps=10, 
                               verbose=1
                               )
print(pred.shape)
pred[:100]

In [ ]:
labels = (val_images.class_indices)
print(labels)
labels = dict((v,k) for k,v in labels.items())  
print(labels)

test_x, test_y = val_images.__getitem__(1)

preds = model.predict(test_x)

plt.figure(figsize=(16, 16))
for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.title('pred:%s / truth:%s' % (labels[np.argmax(preds[i])], labels[np.argmax(test_y[i])]))
    plt.imshow(test_x[i])